In [1]:
import os
import nltk
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from nltk.corpus import stopwords
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)

True

### Create connection to DWH

In [2]:
ENV_URL = os.path.join(os.getcwd(), '../.env')
load_dotenv(ENV_URL)

DWH_NAME = os.environ.get('DWH_NAME')
SERVER_NAME = os.environ.get('SERVER_NAME')
DB_USER = os.environ.get('DB_USER')
DB_PASSWORD = os.environ.get('DB_PASSWORD')

URL = f'mssql+pymssql://{DB_USER}:{DB_PASSWORD}@{SERVER_NAME}/{DWH_NAME}'
URL_LOCAL = f'mssql+pyodbc://{SERVER_NAME}/{DWH_NAME}?trusted_connection=yes&driver=ODBC+Driver+17 for SQL Server'

engine = create_engine(URL_LOCAL)
conn = engine.connect()

### Create a dataframe with all the data from the DWH

In [3]:
def create_query(table_name, columns, condition=None):

    query = f"SELECT "

    for i, column in enumerate(columns):
        if i == 0:
            query += f"[{column}]"
        else:
            query += f", [{column}]"
    
    query += f" FROM [{DWH_NAME}].[dbo].[{table_name}]"
    
    if condition:
        query += f" WHERE {condition}"

    return query


def calc_marketing_pressure(df):
    marketing_pressure_cols = [col for col in df.columns if col.__contains__('persoon_mail_type') 
                           or col.__contains__('persoon_mail_thema') or col.__contains__('persoon_marketingcommunicatie')]

    marketing_pressure_cols.append('bron')
    marketing_pressure_cols.append('visit_first_visit')
    marketing_pressure_cols.append('visit_total_pages')
    marketing_pressure_cols.append('mail_click_freq')

    df['marketing_pressure'] = df[marketing_pressure_cols].sum(axis=1)
    df['marketing_pressure'] = df['marketing_pressure'].astype(int)
    df.drop(marketing_pressure_cols, axis=1, inplace=True)

    return df


def remove_stopwords(text):
    stop_words_nl = set(stopwords.words('dutch'))
    
    word_tokens = word_tokenize(text, language='dutch')

    result = [x for x in word_tokens if x not in stop_words_nl]

    seperator = ', '
    return seperator.join(result)


def team_name_change(text):
    teams_dict = {
        'jo': ' jong ondernemen ',
        'do': ' duurzaam ondernemen ',
        'in': ' innovatie digitalisering ',
        'io': ' internationaal ondernemen ',
        'ao': ' arbeidsmarkt ',
        'ex': ' expert ',
        'gr': ' groei ',
        'bb': ' belangenbehartiging ',
        'co': ' communicatie ',
        'nw': ' netwerking ',
        'ha': ' haven ',
        'ma': ' match '
    }
    word_tokens = word_tokenize(text, language='dutch')
    # apply dict to list
    result = [teams_dict.get(word, word) for word in word_tokens]
    # join list to string
    cleaned_list = ', '.join(result)
    # tokenize string
    tokenize_list = word_tokenize(cleaned_list, language='dutch')
    # remove comma
    tokenize_list_no_comma = [x for x in tokenize_list if x != ',']
    # join list to string and remove duplicates from list
    return ', '.join(list(set(tokenize_list_no_comma)))

#### Account data

In [4]:
acc_cols = ['accountID', 'plaats', 'subregio', 'ondernemingstype', 'ondernemingsaard', 'activiteitNaam']
# account conditie
acc_condition = "accountStatus = 1 AND provincie = 'Oost-Vlaanderen'"
# create query
acc_query = create_query('DimAccount', acc_cols, acc_condition)
# read sql
df_account = pd.read_sql(acc_query, conn)

df_account['plaats'] = df_account['plaats'].str.replace(r'\([a-z.-]+\)', '', regex=True).str.replace('  ', ' ')
df_account['plaats'] = df_account['plaats'] + ' ' + df_account['subregio']

# account ondernemingstype samenvoegen
df_account['onderneming'] = df_account['ondernemingstype'] + ' ' \
                            + df_account['ondernemingsaard'] + ' ' \
                            + df_account['activiteitNaam']

df_account['onderneming'] = df_account['onderneming'].str.replace('unknown', '').str.replace(', , ', '') \
                                                            .str.strip().str.lower() \
                                                            .str.replace(r',$|^,', '', regex=True) \
                                                            .str.replace('&', '').str.replace('-', '')                                                      
# gebruikte kolommen droppen
df_account.drop(columns=['ondernemingstype', 
            'ondernemingsaard', 'activiteitNaam', 'subregio'], inplace=True)

df_account.shape

(3206, 3)

#### Contact data

In [5]:
contact_cols = ['contactID', 'accountID', 'functietitel', 
 'persoon_mail_thema_duurzaamheid', 'persoon_mail_thema_financieel_fiscaal', 'persoon_mail_thema_innovatie',
 'persoon_mail_thema_internationaal_ondernemen', 'persoon_mail_thema_mobiliteit', 'persoon_mail_thema_omgeving',
 'persoon_mail_thema_sales_marketing_communicatie', 'persoon_mail_thema_strategie_en_algemeen_management',
 'persoon_mail_thema_talent', 'persoon_mail_thema_welzijn', 'persoon_mail_type_bevraging', 'persoon_mail_type_communities_en_projecten',
 'persoon_mail_type_netwerkevenementen', 'persoon_mail_type_nieuwsbrieven', 'persoon_mail_type_opleidingen',
 'persoon_mail_type_persberichten_belangrijke_meldingen', 'persoon_marketingcommunicatie',]

contact_condition = "contactStatus = 'Actief'"
contact_query = create_query('DimContact', contact_cols, contact_condition)
df_contact = pd.read_sql(contact_query, conn)

df_contact['functietitel'] = df_contact['functietitel'].str.lower()
df_contact['persoon_marketingcommunicatie'] = df_contact['persoon_marketingcommunicatie'].fillna('-1')
df_contact['persoon_marketingcommunicatie'] = df_contact['persoon_marketingcommunicatie'] \
                                                            .str.replace('Strikt', '0').str.replace('Flexibel', '1') \
                                                            .str.replace('Uitgeschreven', '-1').str.replace('unknown', '-1').astype(int)

df_contact.shape

(396800, 20)

### Merge account and contact data

In [6]:
df_acc_cont = pd.merge(df_contact, df_account, on='accountID', how='inner')
df_acc_cont.shape

(16635, 22)

In [7]:
df_acc_cont['contactID'].nunique()

16635

### Afspraken data

In [8]:
afspraak_cols = ['subthema', 'onderwerp', 'keyphrases', 'contactID']

afspraak_condition = "contactID is not null"
afspraak_query = create_query('DimAfspraak', afspraak_cols, afspraak_condition)
df_afspraak = pd.read_sql(afspraak_query, conn)

df_afspraak['thema'] = df_afspraak['subthema'].str.replace('\(', '', regex=True)

df_afspraak['thema'] = df_afspraak['thema'].str.replace('\)', '', regex=True).str.lower() \
        .str.replace(r'[^\w\s]', '', regex=True).str.replace('  ', ' ').str.strip()

df_afspraak['onderwerp'] = df_afspraak['onderwerp'].str.lower().astype(str).str.replace('ov-', '') \
        .str.replace('ov -', '').str.replace('ov ', '').str.replace('-', ' ') \
        .str.replace(r'[^\w\s]', '', regex=True).str.replace('  ', ' ').str.strip()

df_afspraak['keyphrases'] = df_afspraak['keyphrases'].str.lower().str.replace(r'[^\w\s]', '', regex=True) \
                                                                            .str.replace('  ', ' ').str.strip()

df_afspraak.drop(['subthema'], axis=1, inplace=True)
df_afspraak.drop_duplicates(inplace=True)

df_afspraak.shape

(2547, 4)

In [9]:
afspraak_cols = ['subthema', 'onderwerp', 'keyphrases', 'accountID']

afspraak_condition = "accountID is not null"
afspraak_query = create_query('DimAfspraak', afspraak_cols, afspraak_condition)
df_afspraak1 = pd.read_sql(afspraak_query, conn)

df_afspraak1['thema'] = df_afspraak1['subthema'].str.replace('\(', '', regex=True)

df_afspraak1['thema'] = df_afspraak1['thema'].str.replace('\)', '', regex=True).str.lower() \
        .str.replace(r'[^\w\s]', '', regex=True).str.replace('  ', ' ').str.strip()

df_afspraak1['onderwerp'] = df_afspraak1['onderwerp'].str.lower().astype(str).str.replace('ov-', '') \
        .str.replace('ov -', '').str.replace('ov ', '').str.replace('-', ' ') \
        .str.replace(r'[^\w\s]', '', regex=True).str.replace('  ', ' ').str.strip()

df_afspraak1['keyphrases'] = df_afspraak1['keyphrases'].str.lower().str.replace(r'[^\w\s]', '', regex=True) \
                                                                            .str.replace('  ', ' ').str.strip()

df_afspraak1.drop(['subthema'], axis=1, inplace=True)
df_afspraak1.drop_duplicates(inplace=True)

df_afspraak1.shape

(4861, 4)

### Merge afspraken data met account en contact data

In [10]:
df_acc_cont_afs = df_acc_cont.merge(df_afspraak, on=['contactID'], how='left')
df_acc_cont_afs = df_acc_cont_afs.merge(df_afspraak1, on=['accountID'], how='left')

df_acc_cont_afs['onderwerp_x'] = df_acc_cont_afs['onderwerp_x'].fillna('')
df_acc_cont_afs['onderwerp_y'] = df_acc_cont_afs['onderwerp_y'].fillna('')
df_acc_cont_afs['keyphrases_x'] = df_acc_cont_afs['keyphrases_x'].fillna('')
df_acc_cont_afs['keyphrases_y'] = df_acc_cont_afs['keyphrases_y'].fillna('')
df_acc_cont_afs['thema_x'] = df_acc_cont_afs['thema_x'].fillna('')
df_acc_cont_afs['thema_y'] = df_acc_cont_afs['thema_y'].fillna('')

df_acc_cont_afs['onderwerp'] = df_acc_cont_afs['onderwerp_x'] + ' ' + df_acc_cont_afs['onderwerp_y']
df_acc_cont_afs['keyphrases'] = df_acc_cont_afs['keyphrases_x'] + ' ' + df_acc_cont_afs['keyphrases_y']
df_acc_cont_afs['thema'] = df_acc_cont_afs['thema_x'] + ' ' + df_acc_cont_afs['thema_y']

df_acc_cont_afs['onderwerp'] = df_acc_cont_afs['onderwerp'].str.strip().apply(lambda x: ' '.join(list(set(x.replace('  ', ' ').split(' ')))))
df_acc_cont_afs['keyphrases'] = df_acc_cont_afs['keyphrases'].str.strip().apply(lambda x: ' '.join(list(set(x.replace('  ', ' ').split(' ')))))
df_acc_cont_afs['thema'] = df_acc_cont_afs['thema'].str.strip().apply(lambda x: ' '.join(list(set(x.replace('  ', ' ').split(' ')))))

df_acc_cont_afs.drop(['onderwerp_x', 'onderwerp_y', 'keyphrases_x', 'keyphrases_y', 'thema_x', 'thema_y'], axis=1, inplace=True)
df_acc_cont_afs.drop_duplicates(inplace=True)
df_acc_cont_afs.shape

(31314, 25)

### Campagne data

In [11]:
campagne_cols = ['campagneID', 'campagneNaam', 'campagneType', 'campagneSoort']

campagne_query = create_query('DimCampagne', campagne_cols)
df_campagne = pd.read_sql(campagne_query, conn)

df_campagne['campagneNaam'] = df_campagne['campagneNaam'].str.replace('OV-', '').str.replace('ov-', '') \
                                    .str.replace('-', ' ').str.replace(r'[^\w\s]', '', regex=True) \
                                    .str.replace('  ', ' ').str.strip().str.lower().str.replace('  ', ' ')

df_campagne.shape

(4101, 4)

### Inschrijvingen data

In [12]:
inschrijving_cols = ['campagneID', 'contactID', 'bron']

inschrijving_query = create_query('FactInschrijving', inschrijving_cols)
df_inschrijving = pd.read_sql(inschrijving_query, conn)

df_inschrijving['bron'] = df_inschrijving['bron'].astype(str).str.replace('unknown', '-1') \
                            .str.replace('Website', '0').str.replace('Email', '1').astype(int)

df_inschrijving.shape

(91851, 3)

### Campagne en inschrijvingen data samenvoegen

In [13]:
df_campagne_inschrijving = pd.merge(df_campagne, df_inschrijving, on='campagneID', how='inner')
df_campagne_inschrijving.drop_duplicates(inplace=True)
df_campagne_inschrijving.shape

(48531, 6)

### Sessie data

In [14]:
sessie_cols = ['campaignID', 'themaNaam']

sessie_query = create_query('DimSessie', sessie_cols)
df_sessie = pd.read_sql(sessie_query, conn)

df_sessie = df_sessie.groupby('campaignID')['themaNaam'].apply(list).reset_index()
df_sessie['themaNaam'] = df_sessie['themaNaam'].apply(lambda x: ', '.join(list(set(x))))
df_sessie['themaNaam'] = df_sessie['themaNaam'].str.replace('OV-', '').str.replace('ov-', '') \
                                                            .str.replace('-', ' ').str.replace(r'[^\w\s]', '', regex=True) \
                                                            .str.replace('  ', ' ').str.strip().str.lower().str.replace('  ', ' ')

df_sessie.shape

(1954, 2)

### Sessie en campagne/inschrijvingen data samenvoegen

In [15]:
df_campagne_inschrijving_sessie = pd.merge(df_campagne_inschrijving, df_sessie, left_on='campagneID', right_on='campaignID', how='inner')
df_campagne_inschrijving_sessie.drop(['campaignID', 'campagneID'], axis=1, inplace=True)
df_campagne_inschrijving_sessie.drop_duplicates(inplace=True)
df_campagne_inschrijving_sessie.shape

(48450, 6)

### Campagne, inschrijvingen en sessie data samenvoegen met account en contact data

In [16]:
df_merge = pd.merge(df_acc_cont_afs, df_campagne_inschrijving_sessie, on='contactID', how='left')
df_merge['bron'].fillna(-1, inplace=True)
df_merge.fillna('unknown', inplace=True)
df_merge.shape

(61521, 30)

In [17]:
df_merge['accountID'].nunique(), df_merge['contactID'].nunique()

(3039, 16635)

### Visit data

In [18]:
visit_cols = ['contactID', 'visit_first_visit', 'visit_total_pages', 'mailing_onderwerp', 'mailing_name', 'mailSent_clicks', 'mailSent']

visit_query = create_query('DimVisit', visit_cols)
df_visit = pd.read_sql(visit_query, conn)

df_visit.drop_duplicates(inplace=True)

df_visit['visit_first_visit'] = df_visit['visit_first_visit'] \
      .str.replace('Ja', '0').str.replace('Nee', '1') \
      .str.replace('unknown', '-1').astype(int)

df_visit['visit_total_pages'] = df_visit['visit_total_pages']\
      .replace('unknown', '-1.0').astype(float)

df_visit['aantal_mails'] = df_visit.groupby(
    ['contactID'])['mailSent'].transform('nunique')

df_visit['clicks_total'] = df_visit.groupby(
    ['contactID'])['mailSent_clicks'].transform('sum')

df_visit['visit_total_pages'] = df_visit.groupby(
    ['contactID'])['visit_total_pages'].transform('sum').astype(int)

df_visit['visit_first_visit'] = df_visit.groupby(
    ['contactID'])['visit_first_visit'].transform('sum').astype(int)

df_visit['mailing_name'] = df_visit['mailing_name'].str.replace('OV-', '') \
                                            .str.replace('OV ', '').str.replace('OV -', '') \
                                            .str.replace('-', ' ').str.replace(r'[^\w\s]', ' ', regex=True) \
                                            .str.replace('  ', ' ').str.strip().str.lower().str.replace('  ', ' ')

df_visit['mailing_onderwerp'] = df_visit['mailing_onderwerp'] \
                                            .str.replace('-', ' ').str.replace(r'[^\w\s]', ' ', regex=True) \
                                            .str.replace('  ', ' ').str.strip().str.lower().str.replace('  ', ' ')

df_visit['mail_click_freq'] = np.round(df_visit['clicks_total'] / df_visit['aantal_mails'], 0)
df_visit['mail_click_freq'] = df_visit['mail_click_freq'].fillna(-1).astype(int)

df_visit.drop(['mailSent', 'mailSent_clicks', 'clicks_total', 'aantal_mails'], axis=1, inplace=True)
df_visit.drop_duplicates(inplace=True)
df_visit.shape

(18262, 6)

### Final merge

duurt +- 1.5 minuten

In [19]:
df = pd.merge(df_merge, df_visit, on='contactID', how='left')

df['visit_first_visit'] = df['visit_first_visit'].fillna(-1).astype(int)
df['visit_total_pages'] = df['visit_total_pages'].fillna(-1).astype(int)
df['mail_click_freq'] = df['mail_click_freq'].fillna(-1).astype(int)
df['mailing_onderwerp'] = df['mailing_onderwerp'].fillna('unknown')
df['mailing_name'] = df['mailing_name'].fillna('unknown')

int_cols = df.select_dtypes(include=['int64', 'int32']).columns
df[int_cols] = df[int_cols].astype('int8')

df = calc_marketing_pressure(df)

df['keyphrases'] += ' ' + df['plaats'] + ' ' + df['onderneming'] + ' ' + df['functietitel'] \
                + ' ' + df['onderwerp'] + ' ' + df['thema'] + ' ' + df['campagneNaam'] \
                + ' ' + df['campagneType'] + ' ' + df['campagneSoort'] + ' ' + df['themaNaam'] \
                + ' ' + df['mailing_onderwerp'] + ' ' + df['mailing_name']

df.drop(['plaats', 'onderneming', 'functietitel', 'onderwerp', 'thema', 'campagneNaam', 'campagneType', 'campagneSoort', 
         'themaNaam', 'mailing_onderwerp', 'mailing_name'], axis=1, inplace=True)

df['keyphrases'] = df['keyphrases'].str.replace(', ,', ',').str.replace(r'(\s{2},\s{2}),*+', '').str.replace('unknown', ' ') \
    .str.replace('  ', ' ').str.replace(r'[^\w\s]', '', regex=True).str.replace(r'\d+', '', regex=True) \
    .str.replace('ov', '').str.replace('  ', ' ').str.strip().str.lower() \

df = df.groupby('contactID').agg(list)

df['marketing_pressure'] = df['marketing_pressure'].apply(lambda x: np.mean(x).round(0).astype(int))
df['accountID'] = df['accountID'].apply(lambda x: list(set(x)))
df['accountID'] = df['accountID'].apply(lambda x: x[0])
df['keyphrases'] = df['keyphrases'].apply(lambda x: ', '.join(list(set(x))))
df['keyphrases'] = df['keyphrases'].apply(lambda x: remove_stopwords(x))
df['keyphrases'] = df['keyphrases'].apply(lambda x: team_name_change(x))
df['keyphrases'] = df['keyphrases'].apply(lambda x: ', '.join(sorted(x.split(', '))))
df['marketing_pressure'] = df['marketing_pressure'].apply(lambda x: np.int8(x))

df.reset_index(inplace=True)
df.drop_duplicates(inplace=True)
df.shape

(16635, 4)

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16635 entries, 0 to 16634
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   contactID           16635 non-null  object
 1   accountID           16635 non-null  object
 2   keyphrases          16635 non-null  object
 3   marketing_pressure  16635 non-null  int8  
dtypes: int8(1), object(3)
memory usage: 406.3+ KB


In [21]:
df.head()

,contactID,accountID,keyphrases,marketing_pressure
0,0003747A-026D-EB11-811C-001DD8B72B62,023B965B-C268-E111-B43A-00505680000A,"aanwez, afsprak, band, bedrijf, besliss, besta...",2
1,000A2BB6-B36D-EA11-810F-001DD8B72B62,003B965B-C268-E111-B43A-00505680000A,"aandeelhouderschap, aquafin, bedrijf, bezoek, ...",-4
2,00108666-3B6B-E111-B43A-00505680000A,3B62673D-C268-E111-B43A-00505680000A,"advocat, bedrijf, bereken, chemie, communicati...",5
3,00142269-0B70-E111-B43A-00505680000A,6D4A567D-C368-E111-B43A-00505680000A,"bedrijf, bvba, diensten, erige, gent, industri...",-3
4,0016C5F9-3229-E511-93F3-005056B06EC4,E12937DA-C268-E111-B43A-00505680000A,"aalst, administratiev, advies, afnam, annelies...",27


### Recommendation system

In [75]:
def recommend(df, new_keyphrase: str, top_n=10):
    # preprocessing
    scaler = MinMaxScaler()
    df['marketing_pressure'] = scaler.fit_transform(df[['marketing_pressure']])
    # vectorization
    tfidf = TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform(df['keyphrases'])
    # vectorize the new keyphrase and calculate similarity
    new_keyphrase_tfidf = tfidf.transform([new_keyphrase])
    sim_score_new = cosine_similarity(new_keyphrase_tfidf, tfidf_matrix)
    # sort the similarity scores
    contact_person_similarity = list(enumerate(sim_score_new[0]))
    sorted_contact_persons = sorted(contact_person_similarity, key=lambda x: x[1], reverse=True)
    # get the top n similar contact persons
    top_contact_persons = sorted_contact_persons[:top_n]
    # Create a set to keep track of recommended contact IDs
    recommended_contact_ids = set()
    # Iterate through the sorted contact persons and add unique contact IDs to the set
    for index, _ in top_contact_persons:
        contact_id = df['contactID'][index]
        recommended_contact_ids.add(contact_id)
    # Convert the set back to a list
    recommended_contact_ids = list(recommended_contact_ids)
    # This would not remove the duplicates
    # recommended_contact_ids = [df['contact_contactpersoon_id'][index] for index, _ in top_contact_persons]
    # sort the contact ids by marketing pressure
    recommended_contact_ids = sorted(recommended_contact_ids, key=lambda x: df[df['contactID'] == x]['marketing_pressure'].values[0], reverse=False)
    # result
    results_list = []
    print("Recommended Contact Persons for the New Campaign:")
    for contact_id in recommended_contact_ids:
        marketing_pressure = df[df['contactID'] == contact_id]['marketing_pressure'].values[0]
        results_list.append((contact_id, marketing_pressure))
    return results_list

### Test the recommendation system

In [76]:
campaign_keyphrase = 'volwassen, ov, gent, merelbeke, ondernemen, winst, ceo, bedrijf, netwerkevenement'
recommend(df, campaign_keyphrase)

Recommended Contact Persons for the New Campaign:


[('BF3769A0-C16A-E111-B43A-00505680000A', 0.4716459197786999),
 ('01403D42-3F74-E111-B43A-00505680000A', 0.4716459197786999),
 ('AF84D33D-B06A-E111-B43A-00505680000A', 0.4716459197786999),
 ('5C7FB603-F26E-E111-B43A-00505680000A', 0.4716459197786999),
 ('8B11593F-6EED-E111-9BCA-0050568A003F', 0.47302904564315357),
 ('934C3321-5F71-E111-B43A-00505680000A', 0.4744121715076072),
 ('0870F339-6171-E111-B43A-00505680000A', 0.4744121715076072),
 ('35B35F81-D2DD-E911-8105-001DD8B72B61', 0.4744121715076072),
 ('18663A74-2A70-E111-B43A-00505680000A', 0.4744121715076072),
 ('03BBC819-444C-E711-80E7-001DD8B72B61', 0.48409405255878285)]